In [1]:
#use the intrinio api python wrapper to gather the data
from __future__ import print_function
import time
import intrinio_sdk 
from intrinio_sdk.rest import ApiException
from pprint import pprint
import pandas as pd
import numpy as np
import mysql.connector
import math
import datetime as dt
import sys
import importlib as ib
import GetData as gd

In [2]:
#save and load function
import pickle
def save_obj(obj, name ):
    with open('./'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('./' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
#variable to use

#netdebttoebitda
#ltdebttoebitda
#adj_close
#pricetobook
#adj_volume
#adjdilutedeps
#dividend

In [ ]:
#create a py script to extract all the data needed, save the data in a DataFrame and export them in pkl
#take the price by a 3-month batch
#target is get 6 years of data
#5 year window -> monthly vol
#3 year window -> weekly vol
#1 year window -> daily vol

In [3]:
start_date_test = '2000-01-01' # date | Get historical date on or before this date (optional)
end_date_test = '2019-07-31'
ticker_test = "MSFT"
tags = ['netdebttoebitda','ltdebttoebitda','adj_close_price','pricetobook','adj_volume','adjdilutedeps','dividend']

In [4]:
#reload after changing the script
ib.reload(gd)

<module 'GetData' from 'C:\\Users\\Zijun\\Documents\\Projects\\Volatility Prediction\\GetData.py'>

In [5]:
result_test = gd.get_data(start_date_test,end_date_test,tags,ticker_test,90)

In [6]:
#check length
len(result_test['adj_close_price'])/252

19.944444444444443

In [7]:
#save the result
save_obj(result_test,"pkl_data/result_test_20yr")

In [8]:
result_test = load_obj("pkl_data/result_test_20yr")

In [9]:
result_test['netdebttoebitda']

[{'date': datetime.date(2019, 6, 30), 'value': -1.1611},
 {'date': datetime.date(2019, 3, 31), 'value': -1.1485},
 {'date': datetime.date(2018, 12, 31), 'value': -1.1134},
 {'date': datetime.date(2018, 9, 30), 'value': -1.2509},
 {'date': datetime.date(2018, 6, 30), 'value': -1.2708},
 {'date': datetime.date(2018, 3, 31), 'value': -1.3214},
 {'date': datetime.date(2017, 12, 31), 'value': -1.3674},
 {'date': datetime.date(2017, 9, 30), 'value': -1.4398},
 {'date': datetime.date(2017, 6, 30), 'value': -1.3653},
 {'date': datetime.date(2017, 3, 31), 'value': -1.2497},
 {'date': datetime.date(2016, 12, 31), 'value': -1.2724},
 {'date': datetime.date(2016, 9, 30), 'value': -2.1435},
 {'date': datetime.date(2016, 6, 30), 'value': -2.1763},
 {'date': datetime.date(2016, 3, 31), 'value': -3.3795},
 {'date': datetime.date(2015, 12, 31), 'value': -3.149},
 {'date': datetime.date(2015, 9, 30), 'value': -2.9952},
 {'date': datetime.date(2015, 6, 30), 'value': -2.9957},
 {'date': datetime.date(2015

In [10]:
#turn into dataframe

#first merge price and volume
result_1 = {}

result_1['date'] = [x['date']for x in result_test['adj_close_price']]
result_1['adj_close_price'] = [x['value'] for x in result_test['adj_close_price']]

result_2 = {}
result_2['date'] = [x['date'] for x in result_test['adj_volume']]
result_2['adj_volume'] = [x['value'] for x in result_test['adj_volume']]


result_test_df_1 = pd.DataFrame(result_1)
result_test_df_2 = pd.DataFrame(result_2)

result_test_df = pd.merge(result_test_df_1, result_test_df_2 , left_on='date', right_on='date')

In [11]:
#add price to book
result_3 = {}
result_3['date'] = [x['date'] for x in result_test['pricetobook']]
result_3['pricetobook'] = [x['value'] for x in result_test['pricetobook']]

result_test_df_3 = pd.DataFrame(result_3)

result_test_df = pd.merge(result_test_df, result_test_df_3 , left_on='date', right_on='date')


In [12]:
[x['date'] for x in result_test['netdebttoebitda']][::-1].index(dt.date(2013, 3, 31))

16

In [13]:
#sort it ascendingly
result_test_df = result_test_df.sort_values(by = ['date'])
result_test_df.reset_index(inplace = True)
result_test_df

,index,date,adj_close_price,adj_volume,pricetobook
0,2890,2008-06-30,20.931707,57836460.0,NaN
1,2889,2008-07-01,20.444746,100327707.0,NaN
2,2888,2008-07-02,19.691479,84702804.0,NaN
3,2887,2008-07-03,19.767566,37076426.0,NaN
4,2886,2008-07-07,19.805610,71527599.0,NaN
...,...,...,...,...,...
2886,4,2019-12-02,149.550000,27473616.0,10.8714
2887,3,2019-12-03,149.310000,25192145.0,10.8539
2888,2,2019-12-04,149.850000,17580617.0,10.8932
2889,1,2019-12-05,149.930000,17880601.0,10.8990


In [14]:
#save
save_obj(result_test_df,"pkl_data/result_test_20yr_df")

In [15]:
#load
result_test_df = load_obj("pkl_data/result_test_20yr_df")

In [16]:
#now add the other columns for quarterly data:'netdebttoebitda','ltdebttoebitda','adjdilutedeps','dividend'
tags_loop = ['netdebttoebitda','ltdebttoebitda','adjdilutedeps','dividend']

for t in tags_loop:
    #populate that column with nan
    result_test_df[t] = np.nan
    
    result_tmp = {}
    result_tmp['date'] = [x['date'] for x in result_test[t]][::-1]
    result_tmp[t] = [x['value'] for x in result_test[t]][::-1]
    
    
    #loop through the dataframe and set the value in between the quarterly announcement as the prev one
    for index,row in result_test_df.iterrows():
        #date_in_df = dt.datetime.strptime(row['date'],'%Y-%m-%d')
        date_in_df = row['date']
        #take the previous value for the quarterly data
        #result_tmp['date'] should be sorted
        for dd in result_tmp['date']:
            if dd <= date_in_df:
                #print(dd)
                #print(date_in_df)
                index_tmp = result_tmp['date'].index(dd)
                value_tmp = result_tmp[t][index_tmp]
                result_test_df.loc[index,t] = value_tmp
                #print(row[t])
                #print(index)
                #early exit
                continue

In [17]:
result_test['netdebttoebitda']

[{'date': datetime.date(2019, 6, 30), 'value': -1.1611},
 {'date': datetime.date(2019, 3, 31), 'value': -1.1485},
 {'date': datetime.date(2018, 12, 31), 'value': -1.1134},
 {'date': datetime.date(2018, 9, 30), 'value': -1.2509},
 {'date': datetime.date(2018, 6, 30), 'value': -1.2708},
 {'date': datetime.date(2018, 3, 31), 'value': -1.3214},
 {'date': datetime.date(2017, 12, 31), 'value': -1.3674},
 {'date': datetime.date(2017, 9, 30), 'value': -1.4398},
 {'date': datetime.date(2017, 6, 30), 'value': -1.3653},
 {'date': datetime.date(2017, 3, 31), 'value': -1.2497},
 {'date': datetime.date(2016, 12, 31), 'value': -1.2724},
 {'date': datetime.date(2016, 9, 30), 'value': -2.1435},
 {'date': datetime.date(2016, 6, 30), 'value': -2.1763},
 {'date': datetime.date(2016, 3, 31), 'value': -3.3795},
 {'date': datetime.date(2015, 12, 31), 'value': -3.149},
 {'date': datetime.date(2015, 9, 30), 'value': -2.9952},
 {'date': datetime.date(2015, 6, 30), 'value': -2.9957},
 {'date': datetime.date(2015

In [99]:
#dropna
result_test_df = result_test_df.dropna()

In [18]:
save_obj(result_test_df,"pkl_data/result_test_20yr_df_final")